In [29]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict,Annotated,Literal,List,Optional
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
import operator
import json
import uuid

In [30]:

## Schema For Overall Features from the Story
class MangaFeatureSchema(BaseModel):
    main_characters: List[str] = Field(
        ..., description="List of main characters in the story, including roles or names."
    )
    
    character_descriptions: List[str] = Field(
        ..., description="Short descriptions of the characters’ traits, personalities, or roles."
    )
    
    setting: str = Field(
        ..., description="The primary setting or environment where the story takes place."
    )
    
    conflict_or_goal: str = Field(
        ..., description="The main conflict, tension, or goal driving the story."
    )
    
    important_objects: List[str] = Field(
        ..., description="Key objects, weapons, or magical items relevant to the story."
    )
    
    mood_and_tone: List[
        Literal["dramatic", "mysterious", "adventurous", "romantic", "comedic", "emotional", "dark"]
    ] = Field(
        ..., description="Keywords describing the mood and tone of the story."
    )
    
    key_sound_effects_and_emotions: List[str] = Field(
        ..., description="Important sound effects (onomatopoeia) and strong emotions expressed in the story."
    )




### Schema for Character description
class CharacterProfile(BaseModel):
    name_or_role: str                 # e.g. "Curious Boy"
    canonical_name: Optional[str]     # e.g. "Taro" (or null)
    age_range: str                    # e.g. "early teens (13-15)"
    gender_presentation: Optional[str]# e.g. "male-presenting" or "non-binary"
    body_type: str                    # e.g. "slim, small frame"
    height: Optional[str]             # e.g. "short" or "170 cm"
    face: str                         # short face description: shape, nose, mouth
    hair: str                         # color, style, length
    eyes: str                         # color, shape, notable features
    clothing: str                     # typical outfit description
    accessories: List[str]            # e.g. ["rope belt", "necklace"]
    color_palette: List[str]          # hex or basic color names, ordered primary → accent
    notable_marks: List[str]          # scars, tattoos, birthmarks
    important_objects: List[str]      # items tied to the character, can be []
    signature_poses: List[str]        # short phrases e.g. ["hand-on-hilt", "heroic stance"]
    default_expressions: List[str]    # e.g. ["wide-eyed shock","determined glare"]
    voice_short: Optional[str]        # quick tonal note for dialogue (e.g. "soft, inquisitive")
    drawing_instructions: str         # manga-specific tips: line weight, shading, typical camera angle
    visual_reference_prompt: str      # 1-2 sentence short prompt formatted for image models
    consistency_token: str            # unique id you can pass to image-generator to keep same character

### For list of characters
class CharacterList(BaseModel):
    characters: List[CharacterProfile] = Field(..., description="List of character profiles"
    )




class SceneFeature(BaseModel):
    scene_number: int
    summary: str                                # 1–2 sentence summary
    setting_details: str                        # description of location, mood, time
    characters_involved: List[str]              # must match character_makeup entries
    actions: List[str]                          # short action phrases
    emotions: List[str]                         # emotional keywords
    potential_dialogues: List[str]              # "Name: text"
    inner_thoughts: List[str]                   # [inner thought style]
    sound_effects: List[str]                    # onomatopoeia list


class SceneFeatureList(BaseModel):
    scenes: List[SceneFeature]


## Schema For Director
class Director_Panel(BaseModel):
    panel_number: int
    scene_reference: int                #which scene number this panel is based on",
    scene_description: str              # What is shown in the panel (setting, action, camera)
    characters_present: List[str]       # From character_setup
    actions: List[str]                  # Key actions happening in this panel
    dialogues: List[str]                # Short speech bubbles, "Name: text"
    inner_thoughts: List[str]           # If any, written as [thoughts]
    sound_effects: List[str]            # Onomatopoeia

## Schema for Number of Pages

class MangaPage(BaseModel):
    page_number: int
    panels: List[Director_Panel]



# Schema for each generated image prompt per panel
class MangaImagePrompt(BaseModel):
    panel_number: int = Field(..., description="The panel number from the director script")
    image_prompt: str = Field(
        ..., 
        description="Short, clear description of what the image generation model should draw, including characters (with consistency_token), setting, action, emotions, camera angle, and sound effects"
    )

#  Schema for one manga page of generated prompts
class MangaImagePromptPage(BaseModel):
    page_number: int = Field(..., description="Page number in the manga")
    panel_prompts: List[MangaImagePrompt] = Field(
        ..., description="List of image prompts corresponding to panels on this page"
    )

# # Schema for multiple pages (if extend later)
# class MangaImagePromptBook(BaseModel):
#     pages: List[MangaImagePromptPage] = Field(
#         ..., description="List of pages, each with its panel image prompts"
#     )

In [31]:
_: bool = load_dotenv(find_dotenv())

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")
structured_model_MangaFeature=model.with_structured_output(MangaFeatureSchema)
structured_model_characterList=model.with_structured_output(CharacterList)
structured_model_director=model.with_structured_output(MangaPage)
structured_model_scene=model.with_structured_output(SceneFeatureList)
structured_model_Mangaprompt=model.with_structured_output(MangaImagePromptPage)





In [32]:
class MangaState(TypedDict):
    input_story:str
    refined_story:str
    extracted_features:dict
    character_feature:dict
    scene_features:dict
    panel_scenes:dict
    manga_image_prompts:dict



In [33]:
def prompt_refinner(state:MangaState):
    user_story=state['input_story']
    prompt=f'''
            You are a professional manga storyteller. 
            Your job is to take a short user query and refine it into a concise manga-style story 
            suitable for ONE PAGE comic (4–6 sentences only).

            Requirements:
            - Keep the story short and dynamic (not more than 6 sentences).
            - Add manga-style elements: 
            * Dramatic emotions 
            * Exaggerated action or reactions 
            * Inner thoughts (marked with brackets [ ])
            * Sound effects (onomatopoeia like "BAM!", "WHOOSH!", "Gyaa!")
            - Story should feel like it can naturally be divided into 4–5 panels later.
            - Do not write panel breakdowns yet.

            User Query: {user_story}

            Refined Manga Story:
            '''
    refine_output=model.invoke(prompt).content
    return {"refined_story":refine_output}



def feature_extractor(state:MangaState):
    refine_story=state['refined_story']
    prompt=f"""

            You are a manga story analyzer. 
            Your task is to read the following refined manga story and extract its key features. 
            You MUST return the result as valid JSON that conforms to the MangaFeatureSchema below:

            Schema:
            {{
            "main_characters": ["list of character names or roles"],
            "character_descriptions": ["list of short character descriptions, same order as main_characters"],
            "setting": "short description of where the story takes place",
            "conflict_or_goal": "one-sentence summary of the story’s central conflict or goal",
            "important_objects": ["list of important items, weapons, or artifacts"],
            "mood_and_tone": ["one or more keywords: dramatic, mysterious, adventurous, romantic, comedic, emotional, dark"],
            "key_sound_effects_and_emotions": ["list of notable sound effects (onomatopoeia) and strong emotions"]
            }}

            Rules:
            - Only output valid JSON, no explanations.
            - Keep responses short and concise.
            - Ensure the JSON matches the schema exactly.

            Refined Manga Story:{refine_story}

        """
    output=structured_model_MangaFeature.invoke(prompt)

    return {"extracted_feature":output}


def character_makeup(state: MangaState):
    refined_story = state['refined_story']
    extracted_feature = state['extracted_features']

    # extracted_feature could be a dict or a JSON string depending on prior step
    if isinstance(extracted_feature, dict):
        extracted_feature_json = extracted_feature
    elif isinstance(extracted_feature, str):
        try:
            extracted_feature_json = json.loads(extracted_feature)
        except Exception:
            extracted_feature_json = {"main_characters": [], "character_descriptions": []}
    else:
        extracted_feature_json = {}

    prompt = f'''
        You are a manga character designer. 
        Input: a short refined manga story and the extracted features (characters & brief descriptions).
        Your job: produce a JSON array "characters" of detailed, stable character profiles suitable for repeated drawing across multiple panels.
        You MUST output valid JSON ONLY and match the schema exactly.

        Schema (for each character):
        {{
        "name_or_role": "string",
        "canonical_name": "string or null",
        "age_range": "string",
        "gender_presentation": "string or null",
        "body_type": "string",
        "height": "string or null",
        "face": "short description (shape, nose, mouth, distinguishing facial features)",
        "hair": "short description (color, style, length)",
        "eyes": "short description (color, shape, special details like glow)",
        "clothing": "short description (top, bottom, shoes, texture)",
        "accessories": ["list of accessories"],
        "color_palette": ["primary", "secondary", "accent"],
        "notable_marks": ["scars, tattoos, birthmarks or empty list"],
        "important_objects": ["items associated with this character"],
        "signature_poses": ["list of 2-4 signature poses"],
        "default_expressions": ["list of 3 typical expressions used in manga"],
        "voice_short": "one-line descriptor of speaking voice or null",
        "drawing_instructions": "manga-specific tips (line weight, shading, preferred camera angles)",
        "visual_reference_prompt": "1-2 sentence prompt for an image generator to draw this character consistently",
        "consistency_token": "unique_short_token (use this in downstream image prompts to ensure consistency)"
        }}

        
        Rules:
        - You MUST create one character profile for every entry in "main_characters".
        - The number of profiles in "characters" must exactly equal the number of "main_characters".
        - Use the paired "character_descriptions" to enrich each profile.
        - If details are missing, infer them from the refined story.
        - Keep each profile short, clear, and usable for consistent drawing.
        - Output only valid JSON in the format: {{ "characters": [ ... ] }}

        Refined Story:
        {refined_story}

        Extracted Features:
        {json.dumps(extracted_feature_json)}

'''
    output = structured_model_characterList.invoke(prompt)
    return {"character_feature": output}


def scene_feature_extractor(state: MangaState):
    refined_story = state['refined_story']
    extracted_feature = state['extracted_feature']
    characters = state['character_feature']

    # Normalize inputs
    if isinstance(extracted_feature, dict):
        features_json = extracted_feature
    else:
        try:
            features_json = json.loads(extracted_feature)
        except Exception:
            features_json = {}

    if isinstance(characters, dict):
        characters_json = characters
    else:
        try:
            characters_json = json.loads(characters)
        except Exception:
            characters_json = {}

    prompt = f"""
    You are a Manga Scene Director.  
    Input: a refined short manga story, extracted features, and character profiles.  
    Task: break the story into **4–5 sequential scenes** (not panels yet).  
    These scenes will later guide panel creation.  

    Output Schema (JSON only):
    {{
      "scenes": [
        {{
          "scene_number": 1,
          "summary": "1–2 sentence summary of what happens in this scene",
          "setting_details": "short description of location, mood, time",
          "characters_involved": ["names_or_roles"],
          "actions": ["list of short action phrases"],
          "emotions": ["keywords for emotional tone"],
          "potential_dialogues": ["list of possible dialogue lines (Name: text)"],
          "inner_thoughts": ["list of possible inner thoughts with [brackets]"],
          "sound_effects": ["list of onomatopoeia that could fit this scene"]
        }}
      ]
    }}

    Rules:
    - Always output 4 or 5 scenes. Never fewer.  
    - Each scene should feel like it could become one manga panel later.  
    - Use only characters from the character profiles.  
    - Keep dialogues short, natural, manga-style.  
    - Be consistent with story tone and features.  
    - Return **valid JSON only**.  

    Refined Story:
    {refined_story}

    Extracted Features:
    {json.dumps(features_json)}

    Character Profiles:
    {json.dumps(characters_json)}
    """

    output = structured_model_scene.invoke(prompt)
    return {"scene_features": output}


def manga_director(state: MangaState):
    refined_story = state['refined_story']
    features = state['extracted_feature']
    characters = state['character_feature']
    scenes = state['scene_features']   # ✅ add this

    prompt = f"""
    You are a Manga Director. 
    Your job is to take the refined story, extracted features, character profiles, 
    and pre-extracted scene features, and create a ONE-PAGE manga script divided into **exactly 4–5 panels**.

    Schema:
    {{
      "page_number": 1,
      "panels": [
        {{
          "panel_number": 1,
          "scene_reference": "which scene number this panel is based on",
          "scene_description": "string (describe scene, setting, mood, camera angle)",
          "characters_present": ["list of character names_or_roles"],
          "actions": ["short action phrases"],
          "dialogues": ["Name: text"],
          "inner_thoughts": ["list of inner thoughts if any"],
          "sound_effects": ["list of onomatopoeia like BAM, WHOOSH"]
        }}
      ]
    }}

    Rules:
    - Output **exactly 4 or 5 panels**. Never fewer, never more.
    - Each panel should map to one of the extracted "scenes" (use `scene_reference` field).
    - Use only characters from the given profiles.
    - Keep dialogues short, natural, manga-style.
    - Balance between action, emotion, and pacing.
    - Ensure JSON output only, no extra explanation.

    Refined Story: {refined_story}

    Extracted Features: {features}

    Character Profiles: {characters}

    Scene Features: {scenes}
    """

    output = structured_model_director.invoke(prompt)

    # ✅ enforce 4–5 panels
    if "panels" in output and (len(output["panels"]) < 4 or len(output["panels"]) > 5):
        retry_prompt = prompt + "\n\n⚠️ Reminder: You must output 4–5 panels, not fewer, not more."
        output = structured_model_director.invoke(retry_prompt)

    return {"panel_scenes": output}





def manga_comic_generator(state: MangaState):
    refined_story = state["refined_story"]
    features = state["extracted_feature"]
    characters = state["character_feature"]
    scenes = state["scene_features"]
    panels = state["panel_scenes"]

    prompt = f"""
You are an expert Manga Illustrator AI.
Your task is to generate **manga-style comic panels** based on the story, features, character designs, scene features, and director’s panel instructions.

# Rules for Drawing:
- Style: black-and-white manga style, with clean line art, screentone shading, and dramatic lighting.
- Characters: must remain visually consistent across all panels using their "consistency_token" and "visual_reference_prompt".
- Composition: follow the panel description (camera angle, action, emotion).
- Emotions: exaggerate expressions (wide eyes, sweat drops, speed lines, dramatic shadows).
- Sound Effects: integrate onomatopoeia text (e.g., "BAM!", "WHOOSH!") in stylized manga lettering.
- Dialogues & Inner Thoughts: include all dialogues and inner thoughts from the director’s panel instructions.
- Do not invent new characters or objects outside what’s provided.

# Inputs:
Refined Story:
{refined_story}

Extracted Features:
{json.dumps(features, indent=2)}

Character Profiles (use consistency_token and visual_reference_prompt for each character):
{json.dumps(characters, indent=2)}

Scene Features:
{json.dumps(scenes, indent=2)}

Director’s Panel Script:
{json.dumps(panels, indent=2)}

# Output Instruction:
For each panel in the Director’s Panel Script, generate an **image prompt** formatted as:

{{
  "panel_number": <int>,
  "image_prompt": "<1–3 sentences describing exactly what to draw: characters (with consistency_token), poses, expressions, actions, setting, camera angle, sound effects, and dialogues/inner thoughts>"
}}

Ensure:
- Each panel’s "image_prompt" is short, clear, and suitable for an image generation model.
- Characters must always include their "consistency_token" and and "visual_reference_prompt" for design consistency.
- Include all dialogues and inner thoughts visually in the panel description.
- Use sound effects from the panel if listed.
- Only output valid JSON: a list of panel prompts.
"""

    return {"manga_image_prompts": structured_model_Mangaprompt.invoke(prompt)}


In [34]:
# story=prompt_refinner({'input_story':"A boy name Ibad fall in love with a girl named Aisha."})
# features=feature_extractor({"refined_story":story})
# character_mkp=character_makeup({"refined_story": story["refined_story"], "extracted_features": features["extracted_feature"]})
# scenes_char=scene_feature_extractor({"refined_story": story["refined_story"], "extracted_feature": features["extracted_feature"],"character_feature": character_mkp["character_feature"]})


In [35]:
story='''
Ibad saw Aisha across the crowded schoolyard, and his heart *POW!* skipped a '
 "beat. [She's...an angel!] He clumsily rushed towards her, tripping over his "
 'own feet – *THUD!* Aisha giggled, her laughter like wind chimes. "H-hello!" '
 'Ibad stammered, face burning crimson. Aisha smiled, and Ibad knew, with '
 'absolute certainty, that his life had irrevocably changed. GYAa!')
'''
features=feature_extractor({"refined_story":story})
character_mkp=character_makeup({"refined_story": story, "extracted_features": features["extracted_feature"]})
scenes_char=scene_feature_extractor({"refined_story": story, "extracted_feature": features["extracted_feature"],"character_feature": character_mkp["character_feature"]})


In [36]:
from pprint import pprint

pprint((scenes_char))


{'scene_features': SceneFeatureList(scenes=[SceneFeature(scene_number=1, summary='Ibad spots Aisha in the schoolyard and is immediately smitten.', setting_details='Crowded schoolyard, bright daylight, bustling with students.', characters_involved=['Ibad', 'Aisha'], actions=['Seeing', 'Rushing'], emotions=['Love', 'Excitement', 'Admiration'], potential_dialogues=[], inner_thoughts=["[She's...an angel!]"], sound_effects=['POW!']), SceneFeature(scene_number=2, summary='Ibad clumsily attempts to approach Aisha.', setting_details='Schoolyard, close to Aisha, slightly embarrassing.', characters_involved=['Ibad', 'Aisha'], actions=['Rushing', 'Tripping'], emotions=['Embarrassment', 'Anxiety', 'Hopeful'], potential_dialogues=[], inner_thoughts=[], sound_effects=['THUD!']), SceneFeature(scene_number=3, summary="Aisha giggles at Ibad's clumsiness.", setting_details='Schoolyard, face-to-face, a moment of shared amusement.', characters_involved=['Ibad', 'Aisha'], actions=['Giggling'], emotions=['A

In [37]:

pprint((features['extracted_feature']))

MangaFeatureSchema(main_characters=['Ibad', 'Aisha'], character_descriptions=['Clumsy, lovestruck student', 'Kind, giggling student'], setting='Crowded schoolyard', conflict_or_goal='Ibad tries to talk to Aisha, whom he is infatuated with.', important_objects=[], mood_and_tone=['romantic', 'comedic', 'emotional'], key_sound_effects_and_emotions=['POW!', 'THUD!', 'GYAa!', 'laughter'])


In [38]:

pprint((character_mkp['character_feature']))

CharacterList(characters=[CharacterProfile(name_or_role='Ibad', canonical_name='Ibad', age_range='Teenager', gender_presentation='Male', body_type='Slim', height='Average', face='Round face, slightly upturned nose, kind eyes', hair='Short, messy black hair', eyes='Brown, wide, expressive', clothing='School uniform: slightly oversized blazer, untucked shirt, sneakers', accessories=[], color_palette=['Black', 'White', 'Brown'], notable_marks=[], important_objects=[], signature_poses=['Tripping', 'Stammering with blushing face', 'Looking star-struck'], default_expressions=['Blushing', 'Nervous', 'Amazed'], voice_short='Slightly high-pitched and prone to stammering', drawing_instructions='Use thin lines for hair, emphasize blushing with soft shading', visual_reference_prompt='Draw a teenage boy named Ibad, looking nervous and starstruck. He is wearing a school uniform and has messy black hair.', consistency_token='ibad_001'), CharacterProfile(name_or_role='Aisha', canonical_name='Aisha', a

In [39]:
# director=manga_director({
#     "refined_story": story["refined_story"], 
#     "extracted_feature": features["extracted_feature"],
#     "character_feature": character_mkp["character_feature"],
#     "scene_features":scenes_char["scene_features"]
# })

In [40]:
director=manga_director({
    "refined_story": story, 
    "extracted_feature": features["extracted_feature"],
    "character_feature": character_mkp["character_feature"],
    "scene_features":scenes_char["scene_features"]
})

In [43]:
director

{'panel_scenes': MangaPage(page_number=1, panels=[Director_Panel(panel_number=1, scene_reference=1, scene_description="Wide shot of a crowded schoolyard. Ibad spots Aisha across the yard. Focus on Ibad's surprised expression.", characters_present=['Ibad', 'Aisha'], actions=['Seeing', 'Spotting'], dialogues=[], inner_thoughts=["She's...an angel!"], sound_effects=['POW!']), Director_Panel(panel_number=2, scene_reference=2, scene_description='Close-up of Ibad tripping. Exaggerated comical fall.', characters_present=['Ibad'], actions=['Rushing', 'Tripping'], dialogues=[], inner_thoughts=[], sound_effects=['THUD!']), Director_Panel(panel_number=3, scene_reference=3, scene_description='Aisha giggles, covering her mouth with her hand. Ibad is on the ground, looking up at her.', characters_present=['Aisha', 'Ibad'], actions=['Giggling'], dialogues=[], inner_thoughts=[], sound_effects=[]), Director_Panel(panel_number=4, scene_reference=4, scene_description='Ibad, close-up, face red, stammering.

In [41]:
# manga_comic_generator({
#     "refined_story": story["refined_story"],
#     "extracted_feature": features["extracted_feature"].model_dump(),
#     "character_feature": character_mkp["character_feature"].model_dump(),
#     "scene_features": scenes_char["scene_features"].model_dump(),
#     "panel_scenes": director["panel_scenes"].model_dump()
# })

In [42]:
manga_comic_generator({
    "refined_story": story,
    "extracted_feature": features["extracted_feature"].model_dump(),
    "character_feature": character_mkp["character_feature"].model_dump(),
    "scene_features": scenes_char["scene_features"].model_dump(),
    "panel_scenes": director["panel_scenes"].model_dump()
})

{'manga_image_prompts': MangaImagePromptPage(page_number=1, panel_prompts=[MangaImagePrompt(panel_number=1, image_prompt='Wide shot of a crowded schoolyard. Ibad (ibad_001), draw a teenage boy named Ibad, looking nervous and starstruck. He is wearing a school uniform and has messy black hair., spots Aisha (aisha_002), draw a teenage girl named Aisha, smiling gently. She has long black hair and is wearing a school uniform., across the yard. Ibad\'s eyes are wide with surprise, a thought bubble above his head reads, "She\'s...an angel!" Sound effect: *POW!*'), MangaImagePrompt(panel_number=2, image_prompt='Close-up of Ibad (ibad_001), draw a teenage boy named Ibad, looking nervous and starstruck. He is wearing a school uniform and has messy black hair., tripping and falling forward in an exaggerated, comical way. Motion lines indicate his rapid descent. Sound effect: *THUD!*'), MangaImagePrompt(panel_number=3, image_prompt='Aisha (aisha_002), draw a teenage girl named Aisha, smiling gent